In [1]:
import pandas as pd
import torch as th
from torch import nn
import os
import re
import yfinance as yf
import numpy as np
import warnings
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
warnings.filterwarnings('ignore')
            
import sys; sys.path.insert(0, '..')
from model.models import run_ensemble_strategy, TrainerConfig, Trainer
import data.preprocessing as pp
from utils.utils_analyze import get_price, check_index_dim
from utils.indicators import indicator_list, indicators_stock_stats
from env.BaseEnv import EnvConfig
from env.EnvStock_val import StockEnvValidation
from env.EnvStock_train import StockEnvTrain
from env.EnvStock_trade import StockEnvTrade
from policy.Policies import MlActorCriticPolicy
from config.config import indexes, ticker_list, category_dict, ticker_list_with_dict, tech_tickers
%matplotlib inline
%load_ext autoreload
%autoreload 2

### Do not Run these two cell if you do not want to preprocess data from the begining

In [2]:
START_DATE = '2011-01-01'
SEED = 42069
NORNAMLIZE = True

###  Run these two cell if you do not want to preprocess data from the begining

In [3]:
SAC_PARAMS = {'gamma':0.995,
 'learning_rate':0.016241092434986143,
 'batch_size':64,
 'buffer_size':1000000,
 'learning_starts':10000,
 'gradient_steps':8,
 'ent_coef':'auto',
 'tau':0.005,
 'target_entropy':'auto',
 'policy_kwargs':{'log_std_init': -3.3486909298160947,'net_arch': [256, 256] },
    'use_sde': False,
 'seed':42069,
             "device":"cuda"}


PPO_PARAMS =  {'n_steps': 512, 'batch_size': 64, 'gamma': 0.999, 'learning_rate': 0.03691157097485833, 
                                'ent_coef': 0.08932887565089782, 
                                'clip_range': 0.1, 'n_epochs': 5, 
                                'gae_lambda': 0.9, 'max_grad_norm': 0.8, 'vf_coef': 0.891060944163853, 
                                'policy_kwargs': {'net_arch': [{'pi': [256, 256], 'vf': [256, 256]}], 
              'activation_fn': th.nn.ReLU, 'ortho_init': False}, 'seed': 42069, "device":"cuda"}

REC_PARAMS = {
    "net_arch": [dict(pi=[256, 64, 64], vf=[256, 128])],
        "batch_size": 32,
        "activation_fn": nn.modules.activation.Hardswish,
        "learning_rate":  0.0369,
        "target_step": 100000,
        "lstm_hidden_size":256,
        "n_lstm_layers":2,
        "episode": 2,
        "seed":31,
        "shared_lstm":True
}
DATASET_VERSION = 'pretrainedTrial6'
model = 'reccurent_ppo'
MODEL_NAME = f'jimmy_{model}-v1'
POPULATION = 10
TIME_INTERVAL = 5
START_TRADE = '2011-01-01'
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=[dict(pi=[256, 128, 64], vf=[256, 128])])
TIME_FRAME = 0

In [29]:
winner_params = {'n_steps':8,'batch_size':8,'gamma':0.95,'learning_rate':0.027651081986547765,
 'ent_coef':0.009004220378275581,'clip_range':0.3,'n_epochs':1,'gae_lambda':0.95,'max_grad_norm':0.9,
                 'vf_coef':0.9311987337638572,
 'policy_kwargs':{'net_arch': [{'pi': [256, 256], 'vf': [256, 256]}], 'activation_fn': th.nn.Tanh, 'ortho_init': False},
                 'seed':42069}

In [30]:
params = {'SAC':SAC_PARAMS, 'PPO':PPO_PARAMS, "REC_PPO":REC_PARAMS}

In [40]:
tconfig = TrainerConfig(start_date=START_TRADE,
                        rebalance_window = 126,
                        validation_window = 126,
                        **{'hparams':winner_params},
                        timesteps=50000,
                       policy_kwargs = params['REC_PPO'],
                       index_list = indexes)

In [41]:
trainer = Trainer(model, 'MlpLstmPolicy', 
                  StockEnvTrain, StockEnvValidation, StockEnvTrade, 
                  dataset_version=DATASET_VERSION, population=10,
                     config = tconfig, model_name= MODEL_NAME, debug=False)

{'n_steps': 8, 'batch_size': 8, 'gamma': 0.95, 'learning_rate': 0.027651081986547765, 'ent_coef': 0.009004220378275581, 'clip_range': 0.3, 'n_epochs': 1, 'gae_lambda': 0.95, 'max_grad_norm': 0.9, 'vf_coef': 0.9311987337638572, 'policy_kwargs': {'net_arch': [{'pi': [256, 256], 'vf': [256, 256]}], 'activation_fn': <class 'torch.nn.modules.activation.Tanh'>, 'ortho_init': False}, 'seed': 42069}
Total number of stocks:   469
GPU available


In [ ]:
trainer.train(dataset='datasets/dow50.csv', timesteps=45550, 
              load=False, model_to_load='model_to_load', normalize =NORNAMLIZE)

======Model training from:  2011-01-01 to  2016-01-04T00:00:00.000000000
======Training Agents with the population of 10========
{'n_steps': 8, 'batch_size': 8, 'gamma': 0.95, 'learning_rate': 0.027651081986547765, 'ent_coef': 0.009004220378275581, 'clip_range': 0.3, 'n_epochs': 1, 'gae_lambda': 0.95, 'max_grad_norm': 0.9, 'vf_coef': 0.9311987337638572, 'policy_kwargs': {'net_arch': [{'pi': [256, 256], 'vf': [256, 256]}], 'activation_fn': <class 'torch.nn.modules.activation.Tanh'>, 'ortho_init': False}, 'seed': 42069}
Training time  jimmy_reccurent_ppo-v1 :  10.91240090529124  minutes
======jimmy_reccurent_ppo-v1 Validation from:  2016-01-04 00:00:00 to  2016-07-01 00:00:00
-----------------
Total Reward:  -4.199516787193716
Total Trades:  3343
End total asset for validation 1021915.8873350304
Mean Reward: 0.5318949534761487
STD reward: 0.20680523013041852
-----------------
Reward for the period is -4.199516787193716
Agent #0 has better performance for the training period with total as

2022-10-05 17:55:15.422 ERROR   wandb.jupyter: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Total reward for the the window is 5.176260463951621


wandb: Currently logged in as: kaanb (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


date,▁
end_total_asset,▁
trade_reward,▁
date,1483401600000000000
end_total_asset,1031074.43994
trade_reward,-0.14298


...

Training time  jimmy_reccurent_ppo-v1 :  11.883566570281982  minutes
======jimmy_reccurent_ppo-v1 Validation from:  2020-01-06 00:00:00 to  2020-07-06 00:00:00
-----------------
Total Reward:  -3.0405449941463303
Total Trades:  4964
End total asset for validation 871811.6463035373
Mean Reward: -1.119330525281839
STD reward: 0.04497723332463514
-----------------
Reward for the period is -3.0405449941463303
{'batch_size': 64, 'ent_coef': 0.09734441635468326, 'gae_lambda': 0.99, 'max_grad_norm': 0.3, 'vf_coef': 0.037058848982533865, 'gamma': 0.995, 'learning_rate': 0.04612, 'clip_range': 0.4, 'policy_kwargs': {'net_arch': [{'pi': [256, 64], 'vf': [256, 128]}], 'activation_fn': <class 'torch.nn.modules.activation.Hardswish'>, 'lstm_hidden_size': 64, 'n_lstm_layers': 1, 'shared_lstm': False}, 'seed': 66}
Model destabilized with params:   Creating new params
Expected parameter loc (Tensor of shape (64, 50)) of distribution Normal(loc: torch.Size([64, 50]), scale: torch.Size([64, 50])) to sat

wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


date,▁
end_total_asset,▁
trade_reward,▁
date,1609804800000000000
end_total_asset,1577621.52436
trade_reward,-0.28136


======Model training from:  2011-01-01 to  2020-07-07T00:00:00.000000000
======Training Agents with the population of 10========
{'n_steps': 8, 'batch_size': 8, 'gamma': 0.95, 'learning_rate': 0.027651081986547765, 'ent_coef': 0.009004220378275581, 'clip_range': 0.3, 'n_epochs': 1, 'gae_lambda': 0.95, 'max_grad_norm': 0.9, 'vf_coef': 0.9311987337638572, 'policy_kwargs': {'net_arch': [{'pi': [256, 256], 'vf': [256, 256]}], 'activation_fn': <class 'torch.nn.modules.activation.Tanh'>, 'ortho_init': False}, 'seed': 42069}
Training time  jimmy_reccurent_ppo-v1 :  12.377450466156006  minutes
======jimmy_reccurent_ppo-v1 Validation from:  2020-07-07 00:00:00 to  2021-01-04 00:00:00
-----------------
Total Reward:  5.722559251473285
Total Trades:  3271
End total asset for validation 1148708.8555316483
Mean Reward: 2.396920195611892
STD reward: 0.32857539855802065
-----------------
Reward for the period is 5.722559251473285
Agent #0 has better performance for the training period with total asse

-----------------
Total Reward:  12.35814853780903
Total Trades:  3159
End total asset for validation 1156534.605535051
Mean Reward: 3.3837635717354715
STD reward: 0.22685002741384594
-----------------
Reward for the period is 12.35814853780903
{'batch_size': 128, 'ent_coef': 0.09012089395864403, 'gae_lambda': 0.98, 'max_grad_norm': 0.5, 'vf_coef': 0.12061656519500952, 'gamma': 0.995, 'learning_rate': 0.0118, 'clip_range': 0.2, 'policy_kwargs': {'net_arch': [{'pi': [256, 128, 64], 'vf': [64]}], 'activation_fn': <class 'torch.nn.modules.activation.ReLU'>, 'lstm_hidden_size': 128, 'n_lstm_layers': 2, 'shared_lstm': False}, 'seed': 66}
Training time  jimmy_reccurent_ppo-v1 :  10.135783243179322  minutes
======jimmy_reccurent_ppo-v1 Validation from:  2020-07-07 00:00:00 to  2021-01-04 00:00:00
-----------------
Total Reward:  8.230115237645805
Total Trades:  4994
End total asset for validation 1103057.786502553
Mean Reward: 3.6669952860102057
STD reward: 0.3433416302374492
----------------

In [20]:

pretrain_set = pd.read_csv('datasets/pretrain_set3.csv')
pretrain_set['turbulence'] = 0

In [ ]:
pretrain = trainer.pretrain(sn, '2011-05-01', START_TRADE, NORNAMLIZE)

In [67]:
model_to_load = pretrain.split('/')[1]

In [ ]:
trainer.cluster(False, period=365, number_of_clusters=5, stocks_per_cluster=5)

In [60]:
m = pd.read_csv('./datasets/.csv')

TypeError: list indices must be integers or slices, not tuple